In [1]:
import torch
from simpletransformers.ner import NERModel, NERArgs

_NER_TAGS = ["O", "B_C", "B_P", "I_C", "I_P"]
ner_args = NERArgs()
ner_args.max_seq_length = 512
test_ner = NERModel("bert", 'model_seq_512/save_model', args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS)

In [2]:
def tag_html_format(predict_list, pos=False): # get ist of tuple 1 sentent [(word, pos, tag), .....]
    text_result = ""
    label_start = ""
    start_tag = False
    tag_label = ""
        
    for token in predict_list: # list of tuple
        if pos == True:
            word = token[0]
            tag = token[2]
        else:
            word = token[0]
            tag = token[1]
        
        if tag == "O":
            if start_tag == True :
                label_end = "</claim>" if label_start == "<claim>" else "</premise>"
                text_result += label_end
                text_result += word
                start_tag = False
            else:
                text_result += word
        else:
            if start_tag == False:
                tag_label = tag.split("-")[1]  #I-c  = c 
                label_start = "<claim>" if tag_label == "c" else "<premise>"
                text_result += label_start
                text_result += word
                start_tag = True
            else:
                if tag_label != tag.split("-")[1]: #กรณีที่tag ต่างกันอยู่ติดกัน
                    label_end = "</claim>" if label_start == "<claim>" else "</premise>"
                    text_result += label_end
                    tag_label = tag.split("-")[1]  #I-c  = c 
                    label_start = "<claim>" if tag_label == "c" else "<premise>"
                    text_result += label_start
                    text_result += word
                    start_tag = True
                else:
                    text_result += word
     
    if start_tag == True:
        label_end = "</claim>" if label_start == "<claim>" else "</premise>"
        text_result += label_end
                
    return text_result

In [3]:
def predict_text(text):
    predict_text = []
    text_token = word_tokenize(text)
    predictions, raw_outputs = test_ner.predict([text_token], split_on_space=False)
    # convert [{}] to [()]
    # convert B_C to B-c
    for pred_dict in predictions[0]:
        for word, label in pred_dict.items():
             label = label.capitalize().replace('_', '-')
             predict_text.append((word, label))
    return predict_text, tag_html_format(predict_text)

In [4]:
from pythainlp.tokenize import word_tokenize

text = "การใช้ถุงพลาสติดแทนถุงผ้าผมคิดว่ามันจะทำให้ช่วยลดโลกร้อนได้มากขึ้น"

pred_text, pred_tag = predict_text(text)
print(pred_text)
print("----")
print(pred_tag)

Running Prediction: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]

[('การ', 'B-c'), ('ใช้', 'I-c'), ('ถุง', 'I-c'), ('พ', 'I-c'), ('ลา', 'I-c'), ('สติ', 'I-c'), ('ด', 'I-c'), ('แทน', 'I-c'), ('ถุง', 'I-c'), ('ผ้า', 'I-c'), ('ผม', 'I-c'), ('คิด', 'I-c'), ('ว่า', 'I-p'), ('มัน', 'I-p'), ('จะ', 'I-p'), ('ทำให้', 'I-p'), ('ช่วย', 'I-p'), ('ลด', 'I-p'), ('โลก', 'I-p'), ('ร้อน', 'I-p'), ('ได้', 'I-p'), ('มากขึ้น', 'I-p')]
----
<claim>การใช้ถุงพลาสติดแทนถุงผ้าผมคิด</claim><premise>ว่ามันจะทำให้ช่วยลดโลกร้อนได้มากขึ้น</premise>
